# Webscrapeo : Ubica tu casilla

### Paquetes

In [222]:
import time
import requests
import numpy as np
import pandas as pd

### Pipeline

In [230]:
API = "https://us-central1-ine-ubica-tu-casilla.cloudfunctions.net/utc-api/search/rasgo-with-cmr-uts?e=1&s=488"

headers = {"authority": "us-central1-ine-ubica-tu-casilla.cloudfunctions.net",
            "method": "GET",
            "scheme": "https",
            "accept": "application/json, text/plain, */*",
            "accept-encoding": "gzip, deflate, br",
            "origin": "https://ubicatucasilla.ine.mx",
            "referer": "https://ubicatucasilla.ine.mx/",
            "accept-language": "en,es-419;q=0.9,es;q=0.8",
            "user-agent": "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/100.0.4896.75 Safari/537.36",
            "x-utc-service-id": "ubicatucasilla",
            "x-utc-version-id": "2022",
            "sec-ch-ua": 'Not A;Brand";v="99", "Chromium";v="100", "Google Chrome";v="100"',
            "sec-ch-ua-mobile": "?0",
            "sec-ch-ua-platform": "Windows",
            "sec-fetch-dest": "empty",
            "sec-fetch-mode": "cors",
            "sec-fetch-site": "cross-site",
}

In [258]:
def get_raw_df(API, num_edo, num_casilla):
    """
    Extrae el dataframe de una casilla a partir del API
     
    Parameters
    API: string
        URL del API
    num_edo: int
        Numero de estado
    num_casilla: int
        Numero de casilla a extraer
    """
    path = "/utc-api/search/rasgo-with-cmr-uts?e={num_edo}&s={num_casilla}"
    headers["path"] = path 

    new_API = API.replace("e=1", f"e={num_edo}")
    new_API = new_API.replace("s=488", f"s={num_casilla}")
    r = requests.get(new_API, headers=headers)
    
    if r.json()["status"] == False:
        raw_df = pd.DataFrame([])
        if 'cmr' in r.json().keys():
            content = r.json()

        else:
            raw_df = pd.DataFrame([])
            content = {}
    
    else:
        content = r.json()['data'][0]['items']
        raw_df = pd.DataFrame(content[0]['apellidos'])
        raw_df['domicilio'] = content[0]['data']['domicilio']
        raw_df['ubicacion'] = content[0]['data']['ubicacion']

        raw_df['entidad'] = content[0]['entidadSede']
        raw_df['distitoSede'] = content[0]['distritoSede']
        raw_df['distritoLocal'] = content[0]['distrito_localSede']
        
        raw_df 

    return raw_df, content

In [259]:
def get_clean_df(raw_df, content):
    """
    Saca clean_df a partir del raw_df y pone varios 
    dfs de acuerdo al numero de secciones
    
    Parameters
    ----------
    raw_df: dataframe
        Dataframe de la casilla
    content: dict
        Diccionario con la informacion de la casilla
    
    """
    raw_secciones = content[0]['secciones']

    clean_secciones = [int(x) for x in raw_secciones]
    complete_df = pd.DataFrame([])

    for clean_seccion in clean_secciones:
        copy_df = raw_df.copy()
        copy_df['seccion'] = clean_seccion

        complete_df = pd.concat([complete_df, copy_df], 
                                ignore_index=True)

    return complete_df, clean_secciones

In [275]:
def extract_estado(API, num_edo):
    """
    Extrae todas las casillas de un estado

    Parameters
    ----------
    API : string
        API de INE
    num_edo : int
        Numero de estado a extraer

    Returns
    -------
    estado_df : dataframe
        Dataframe con todas las casillas de un estado
    pending_sections : list
        Lista con todas las secciones pendientes
    """

    all_secciones = np.arange(1, 7000)
    extracted_sections = []
    estado_df = pd.DataFrame([])
    pending_sections = []
    counter = 0
    non_existent_counter = 0

    for seccion in all_secciones:
        counter += 1
        if counter % 100 == 0:
            print(f"Extracted {counter} casillas")

        # Si la seccion ya la extrajimos
        if seccion in extracted_sections:
            continue

        raw_df, content = get_raw_df(API, num_edo, seccion)

        if content:

            if not isinstance(content, list):
                time.sleep(3)
                pending_sections.append(seccion)
                # Si falla el API, continuamos
                continue
            else:
                # Sacar datos limpios y quitar casillas que ya hayamos extraido
                complete_df, clean_secciones = get_clean_df(raw_df, content)
                estado_df = pd.concat([estado_df, complete_df], ignore_index=True)
                # Agregar las secciones ya extraidas
                extracted_sections.extend(clean_secciones)

        # Si el content está vacío, continuamos
        else:
            non_existent_counter += 1
            
            if non_existent_counter >= 40:
                print("40 casillas no encontradas, stopping...")
                break
            else:
                continue
    
    return estado_df, pending_sections

### Pipeline

In [276]:
estados = np.arange(1, 32)
mexico_df = pd.DataFrame([])
pendientes_dic = {}

for estado in estados:

    estado_df, pending_sections = extract_estado(API, estado)
    pendientes_dic[estado] = pending_sections

    mexico_df = pd.concat([mexico_df, estado_df], ignore_index=True)

    print(f"========== Extracted {estado} ===========")
    time.sleep(5)
        

Extracted 100 casillas
Extracted 200 casillas
Extracted 300 casillas
Extracted 400 casillas
Extracted 500 casillas
Extracted 600 casillas
40 casillas no encontradas, stopping...
========== Extracted 1 ===========
Extracted 100 casillas
Extracted 200 casillas
Extracted 300 casillas


In [274]:
mexico_df

,casilla,inicial,final,domicilio,ubicacion,entidad,distitoSede,distritoLocal,seccion
0,B,ABOYTES DOÑAS,LLAMAS DELGADO,"CALLE HUIZACHE, NÚMERO 108, FRACCIONAMIENTO EL...",DOMICILIO PARTICULAR,1,3,6,1
1,C1,LLAMAS ESPERON,ZUÑIGA GONZALEZ,"CALLE HUIZACHE, NÚMERO 108, FRACCIONAMIENTO EL...",DOMICILIO PARTICULAR,1,3,6,1
2,B,ABREGO GUTIERREZ,LLAMAS VAZQUEZ,"AVENIDA GUADALUPE GONZÁLEZ, SIN NÚMERO, COLONI...",AULA MAGNA DEL DESARROLLO INTEGRAL DE LA FAMIL...,1,3,6,2
3,C1,LOERA CHAVEZ,ZURITA ALVARADO,"AVENIDA GUADALUPE GONZÁLEZ, SIN NÚMERO, COLONI...",AULA MAGNA DEL DESARROLLO INTEGRAL DE LA FAMIL...,1,3,6,2
4,B,ABREGO GUTIERREZ,LLAMAS VAZQUEZ,"AVENIDA GUADALUPE GONZÁLEZ, SIN NÚMERO, COLONI...",AULA MAGNA DEL DESARROLLO INTEGRAL DE LA FAMIL...,1,3,6,4
...,...,...,...,...,...,...,...,...,...
7940,C1,LEON GONZALEZ,ZURITA TRUJILLO,CALLE MESÓN ENTRE CAMINO DEL TRIUNFO Y ANTONIO...,CENTROS ESCOLARES PROFESOR CÉSAR PIÑEDA CHACÓN,3,1,4,493
7941,B,ABUNDIS SOTELO,LIZARRAGA RODRIGUEZ,CALLE CAMINO DE MIRAFLORES ESQUINA CON CALLE P...,CANCHA PÚBLICA MUNICIPAL,3,1,4,494
7942,C1,LOBATO GRACIANO,ZURITA QUIJADA,CALLE CAMINO DE MIRAFLORES ESQUINA CON CALLE P...,CANCHA PÚBLICA MUNICIPAL,3,1,4,494
7943,B,ABUNDIS SOTELO,LIZARRAGA RODRIGUEZ,CALLE CAMINO DE MIRAFLORES ESQUINA CON CALLE P...,CANCHA PÚBLICA MUNICIPAL,3,1,4,495


In [257]:
estado_df.to_csv("casillas_aguascalientes.csv", index=False,
                encoding='utf-8-sig')

In [246]:
raw_df, content = get_raw_df(API, 1, 98)

In [247]:
content

{}

In [228]:
content

{'status': False, 'cmr': {'entidad': 1, 'seccion_dice': 81, 'flag': 3}}

In [195]:
np.random.randint(1,32)

19

In [243]:
pending_sections

[]

In [242]:
extracted_sections

[1,
 2,
 4,
 5,
 6,
 3,
 8,
 9,
 7,
 11,
 12,
 13,
 14,
 15,
 10,
 17,
 18,
 16,
 21,
 22,
 23,
 19,
 20,
 33,
 34,
 24,
 25,
 26,
 27,
 30,
 28,
 71,
 72,
 73,
 74,
 172,
 29,
 61,
 62,
 63,
 65,
 66,
 67,
 31,
 32,
 35,
 36,
 272,
 37,
 38,
 39,
 40,
 171,
 173,
 174,
 41,
 42,
 43,
 50,
 51,
 44,
 45,
 46,
 47,
 48,
 49,
 52,
 53,
 55,
 54,
 60,
 56,
 57,
 58,
 59,
 64,
 106,
 107,
 108,
 109,
 68,
 69,
 70,
 78,
 75,
 76,
 77,
 79,
 181,
 182,
 183,
 80,
 82,
 83,
 84,
 85,
 86,
 87,
 88,
 94,
 89,
 93,
 95,
 102,
 103,
 104,
 105,
 90,
 91,
 92,
 96,
 97,
 100,
 101]

In [163]:
estado_df

,casilla,inicial,final,domicilio,ubicacion,entidad,distitoSede,distritoLocal,seccion
0,B,ABOYTES DOÑAS,LLAMAS DELGADO,"CALLE HUIZACHE, NÚMERO 108, FRACCIONAMIENTO EL...",DOMICILIO PARTICULAR,1,3,6,1
1,C1,LLAMAS ESPERON,ZUÑIGA GONZALEZ,"CALLE HUIZACHE, NÚMERO 108, FRACCIONAMIENTO EL...",DOMICILIO PARTICULAR,1,3,6,1
2,B,ABREGO GUTIERREZ,LLAMAS VAZQUEZ,"AVENIDA GUADALUPE GONZÁLEZ, SIN NÚMERO, COLONI...",AULA MAGNA DEL DESARROLLO INTEGRAL DE LA FAMIL...,1,3,6,2
3,C1,LOERA CHAVEZ,ZURITA ALVARADO,"AVENIDA GUADALUPE GONZÁLEZ, SIN NÚMERO, COLONI...",AULA MAGNA DEL DESARROLLO INTEGRAL DE LA FAMIL...,1,3,6,2
4,B,ABREGO GUTIERREZ,LLAMAS VAZQUEZ,"AVENIDA GUADALUPE GONZÁLEZ, SIN NÚMERO, COLONI...",AULA MAGNA DEL DESARROLLO INTEGRAL DE LA FAMIL...,1,3,6,4
...,...,...,...,...,...,...,...,...,...
206,C1,LOPEZ RIVERA,ZOZAYA ALVAREZ,"AVENIDA ALAMEDA, NÚMERO 206, BARRIO DE LA ESTA...",CENTRO CULTURAL LOS ARQUITOS,1,3,11,183
207,B,ABARCA ESPINOSA,ESPINO SANCHEZ,"CALLE ARTÍCULO 31, SIN NÚMERO, FRACCIONAMIENTO...",ESCUELA PRIMARIA SATURNINO HERRÁN,1,2,5,80
208,C1,ESPINOSA ALONSO,LUCERO REYES,"CALLE ARTÍCULO 31, SIN NÚMERO, FRACCIONAMIENTO...",ESCUELA PRIMARIA SATURNINO HERRÁN,1,2,5,80
209,C2,LUCIO CHAVEZ,RAMOS AVILA,"CALLE ARTÍCULO 31, SIN NÚMERO, FRACCIONAMIENTO...",ESCUELA PRIMARIA SATURNINO HERRÁN,1,2,5,80


In [248]:
raw_df, content = get_raw_df(API, 1, 98)

In [249]:
new_API = API.replace("e=1", f"e={1}")
new_API = new_API.replace("s=488", f"s={98}")
r = requests.get(new_API, headers=headers, proxies=proxies)
r.json()

{'status': False, 'message': "Cannot read property 'status' of undefined"}